# OpenSearch MCP Server

This notebook demonstrates how to set up and use OpenSearch MCP server in both stdio and SSE modes, including server configuration and tool execution. For complete documentation, see [opensearch-mcp-server-py](https://github.com/opensearch-project/opensearch-mcp-server-py) repository.

## Stdio Server

Configure MCP server with `stdio` protocol using the sample below. This configuration works with both [Claude Desktop](https://modelcontextprotocol.io/quickstart/user) and [Q Developer CLI](https://github.com/aws/amazon-q-developer-cli). For more detailed information, see the OpenSearch MCP server [user guide](https://github.com/opensearch-project/opensearch-mcp-server-py/blob/main/USER_GUIDE.md#quick-start).

```json
{
    "mcpServers": {
        "opensearch-mcp-server": {
            "command": "uvx",
            "args": [
                "opensearch_mcp_server_py"
            ],
            "env": {
                // Optional
                "OPENSEARCH_URL": "<your_opensearch_domain_url>",
                
                // For OpenSearch Serverless
                "AWS_OPENSEARCH_SERVERLESS": "true",  // Set to "true" for OpenSearch Serverless

                // For Basic Authentication
                "OPENSEARCH_USERNAME": "<your_opensearch_domain_username>",
                "OPENSEARCH_PASSWORD": "<your_opensearch_domain_password>",

                // For IAM Role Authentication
                "AWS_REGION": "<your_aws_region>",
                "AWS_ACCESS_KEY_ID": "<your_aws_access_key>",
                "AWS_SECRET_ACCESS_KEY": "<your_aws_secret_access_key>",
                "AWS_SESSION_TOKEN": "<your_aws_session_token>"
            }
        }
    }
}
```

## SSE Server

Follow these steps to set up MCP server with `sse` protocol using OpenSearch MCP client:

### 1. Install `opensearch-mcp-server-py` package

Opensearch-mcp-server-py can be installed from [PyPI](https://pypi.org/project/opensearch-mcp-server-py/) via pip:

In [ ]:
%pip install opensearch-mcp-server-py

### 2. Set preferred authentication

Both basic authentication and IAM authentication can be configured via either global environment variables or environment variables in agent MCP config file.

#### a. Basic Authentication

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['OPENSEARCH_USERNAME'] = "<your_opensearch_domain_username>"
os.environ['OPENSEARCH_PASSWORD'] = "<your_opensearch_domain_password>"

#### b. IAM Role Authentication

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['AWS_REGION'] = "<your_aws_region>"
os.environ['AWS_ACCESS_KEY_ID'] = "<your_aws_access_key>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<your_aws_secret_access_key>"
os.environ['AWS_SESSION_TOKEN'] = "<your_aws_session_token>"

#### c. OpenSearch Serverless

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['AWS_OPENSEARCH_SERVERLESS'] = True
os.environ['AWS_REGION'] = "<your_aws_region>"
os.environ['AWS_ACCESS_KEY_ID'] = "<your_aws_access_key>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<your_aws_secret_access_key>"
os.environ['AWS_SESSION_TOKEN'] = "<your_aws_session_token>"

### 3. Run the server

By default, the server will run on port **9900**. You can change the port number by specifying the port number in below command, for example:

`["python", "-m", "mcp_server_opensearch", "--transport", "sse", "--port", "<your_desired_port_number>"]`

In [ ]:
import subprocess
process = subprocess.Popen(
    ["python", "-m", "mcp_server_opensearch", "--transport", "sse"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

> Note: The server will keep running in the background, so you can continue using the notebook.

### 4. Create an MCP connector

An MCP connector stores connection details and credentials for your MCP server.

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# Change these variables with your OpenSearch and MCP server information
opensearch_domain_url = "<your_opensearch_domain_url>"
opensearch_domain_username = "<your_opensearch_domain_username>"
opensearch_domain_password = "<your opensearch_domain_password"
mcp_server_url = "<your_mcp_server_url>"  # For example "http://localhost:9900"
mcp_server_api_key = "<your_mcp_server_api_key>"  # For example "secret-token"

# Configure settings needed to enable MCP protocol
payload = {
  "persistent": {
    "plugins.ml_commons.trusted_connector_endpoints_regex": [
        mcp_server_url,
        "https://api.openai.com/v1/chat/completions"
    ],
    "plugins.ml_commons.mcp_connector_enabled": "true"
  }
}
settings_response = requests.put(f'{opensearch_domain_url}/_cluster/settings',
                        auth=HTTPBasicAuth(opensearch_domain_username, opensearch_domain_password),
                        json=payload,
                        headers={"Content-Type": "application/json"})
print(settings_response.text)

# Create an MCP connector
payload = {
    "name": "My MCP Connector",
    "description": "Connects to the OpenSearch MCP server",
    "version": 1,
    "protocol": "mcp_sse",
    "url": mcp_server_url,
    "credential": {
        "mcp_server_key": mcp_server_api_key
    },
    "headers": {
        "Authorization": "Bearer ${credential.mcp_server_key}"
    }
}
connector_response = requests.post(f'{opensearch_domain_url}/_plugins/_ml/connectors/_create',
                        auth=HTTPBasicAuth(opensearch_domain_username, opensearch_domain_password),
                        json=payload,
                        headers={"Content-Type": "application/json"})
print(connector_response.text)

### 5. Register a remote model

Register any externally hosted large language model (LLM) using a connector. For a list of supported models, see [Supported connectors](https://docs.opensearch.org/docs/latest/ml-commons-plugin/remote-models/supported-connectors/).

In this notebook, we're using OpenAI chat model, but you can register any model.

In [ ]:
# Change this variable with your model API key
openai_api_key = "<your_openai_api_key>"

# Create OpenAI chat model
payload = {
    "name": "My OpenAI model: gpt-4",
    "function_name": "remote",
    "description": "Test model registration",
    "connector": {
        "name": "My OpenAI Connector: gpt-4",
        "description": "Connector for the OpenAI chat model",
        "version": 1,
        "protocol": "http",
        "parameters": {
            "model": "gpt-4o"
        },
        "credential": {
            "openAI_key": openai_api_key
        },
        "actions": [{
            "action_type": "predict",
            "method": "POST",
            "url": "https://api.openai.com/v1/chat/completions",
            "headers": {
                "Authorization": "Bearer ${credential.openAI_key}"
            },
            "request_body": "{ \"model\": \"${parameters.model}\", \"messages\": [{\"role\":\"developer\",\"content\":\"${parameters.system_instruction}\"},${parameters._chat_history:-}{\"role\":\"user\",\"content\":\"${parameters.prompt}\"}${parameters._interactions:-}], \"tools\": [${parameters._tools:-}],\"parallel_tool_calls\":${parameters.parallel_tool_calls},\"tool_choice\": \"${parameters.tool_choice}\" }"
        }]
    }
}
response = requests.post(f'{opensearch_domain_url}/_plugins/_ml/models/_register',
                        auth=HTTPBasicAuth(opensearch_domain_username, opensearch_domain_password),
                        json=payload,
                        headers={"Content-Type": "application/json"})
print(response.text)

### 6. Register an agent for accessing MCP tools

Currently, MCP tools can only be used with [conversational](https://docs.opensearch.org/docs/latest/ml-commons-plugin/agents-tools/agents/conversational/) or [plan-execute-reflect](https://docs.opensearch.org/docs/latest/ml-commons-plugin/agents-tools/agents/plan-execute-reflect/) agent types.

In this notebook, we'll use conversational agent.

In [ ]:
# Change these variables with the connector ID and model ID from previous steps
mcp_connector_id = "<your_mcp_connector_id>"  # From step 4
llm_model_id = "<your_llm_model_id>"  # From step 5

# Register a conversational agent
payload = {
    "name": "OpenSearch MCP server",
    "type": "conversational",
    "description": "Uses MCP to information related to OpenSearch",
    "llm": {
        "model_id": llm_model_id,
        "parameters": {
            "max_iteration": 5,
            "system_instruction": "You are a helpful assistant.",
            "prompt": "${parameters.question}"
        }
    },
    "memory": {
        "type": "conversation_index"
    },
    "parameters": {
        "_llm_interface": "openai/v1/chat/completions",
        "mcp_connectors": [{
            "mcp_connector_id": mcp_connector_id
        }]
    },
    "app_type": "os_chat"
}
response = requests.post(f'{opensearch_domain_url}/_plugins/_ml/agents/_register',
                        auth=HTTPBasicAuth(opensearch_domain_username, opensearch_domain_password),
                        json=payload,
                        headers={"Content-Type": "application/json"})
print(response.text)

### 7. Execute the agent

Example questions you can ask the agent about OpenSearch:
- `ListIndexTool` : List all indices in my OpenSearch cluster?
- `GetShardsTool` : Can you get the shards information of an index called .plugins-ml-config in my OpenSearch cluster?
- `ClusterHealthTool` : Can you check my OpenSearch cluster health?
- `SearchIndexTool` : I have an index called sample_data_ecommerce, can you show me the email addresses of 20 customers from this index?

> See more available tools in [OpenSearch MCP server](https://github.com/opensearch-project/opensearch-mcp-server-py/blob/main/README.md#available-tools) repository. 

In [ ]:
# Change these variables with the agent ID from previous step and the question for the agent
agent_id = "<your_agent_id>"  # From step 6
question = "<your_question>"

# Execute the agent
payload = {
  "parameters": {
    "question": question,
    "verbose": True
  }
}
response = requests.post(f'{opensearch_domain_url}/_plugins/_ml/agents/{agent_id}/_execute',
                        auth=HTTPBasicAuth(opensearch_domain_username, opensearch_domain_password),
                        json=payload,
                        headers={"Content-Type": "application/json"})
print(response.text)